In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import  MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle
import gc

# Data
label I1-I13 C1-C26

In [2]:
data = pd.read_csv('../../data/criteo_data/train.txt', sep='\t', header=None)
y = data[0]
X = data.drop([0], axis=1)

del data
gc.collect()

22

In [3]:
continuous_feature = list(range(1,14))
category_feature = list(range(14, 40))
X[continuous_feature] = X[continuous_feature].fillna(0, )
X[category_feature] = X[category_feature].fillna('-1', )
mms = MinMaxScaler(feature_range=(0, 1))
X[continuous_feature] = mms.fit_transform(X[continuous_feature])

In [4]:
X_trian, X_valid, y_train, y_valid = train_test_split(X, y)

In [5]:
class DataParse:
    def __init__(self, category_feature, continuous_feature, ignore_feature=[], feature_dict={}, feature_size=0, field_size=0):
        self.feature_dict = feature_dict
        self.feature_size = feature_size
        self.field_size = field_size
        self.ignore_feature = ignore_feature
        self.category_feature = category_feature
        self.continuous_feature = continuous_feature
    
    def FeatureDictionary(self, train, test):
        """
        目的是给每一个特征维度都进行编号。
        1. 对于离散特征，one-hot之后每一列都是一个新的特征维度(计算编号时，不算0)。所以，原来的一维度对应的是很多维度，编号也是不同的。
        2. 对于连续特征，原来的一维特征依旧是一维特征。
        返回一个feat_dict，用于根据原特征名称和特征取值 快速查询出 对应的特征编号。
        train: 原始训练集
        test:  原始测试集
        continuous_feature: 所有数值型特征
        ignore_feature: 所有忽略的特征. 除了数值型和忽略的，剩下的全部认为是离散型
        feat_dict, feat_size
             1. feat_size: one-hot之后总的特征维度。
             2. feat_dict是一个{}， key是特征string的col_name, value可能是编号（int），可能也是一个字典。
             如果原特征是连续特征： value就是int，表示对应的特征编号；
             如果原特征是离散特征：value就是dict，里面是根据离散特征的 实际取值 查询 该维度的特征编号。 因为离散特征one-hot之后，
             一个取值就是一个维度，而一个维度就对应一个编号。
        """
        df = pd.concat([train, test], axis=0)
        feat_dict = {}
        total_cnt = 0
        
        for col in df.columns:
            # 连续特征只有一个编号
            if col in self.continuous_feature:
                feat_dict[col] = total_cnt
                total_cnt = total_cnt + 1
            elif col in self.category_feature:
                unique_vals = df[col].unique()
                unique_cnt = df[col].nunique()
                feat_dict[col] = dict(zip(unique_vals, range(total_cnt, total_cnt + unique_cnt)))
                total_cnt = total_cnt + unique_cnt
        
        self.feature_size = total_cnt
        self.feature_dict = feat_dict
        print('feat_dict=', feat_dict)
        print('feature_size=', total_cnt)
    
    def parse(self, df):
        dfi = df.copy()
        dfv = df.copy()
        for col in dfi.columns:
            if col in self.ignore_feature:
                dfi.drop([col], axis=1, inplace=True)
                dfv.drop([col], axis=1, inplace=True)

            elif col in self.continuous_feature:  # 连续特征1个维度，对应1个编号，这个编号是一个定值
                dfi[col] = self.feature_dict[col]

            elif col in self.category_feature:  # 离散特征。不同取值对应不同的特征维度，编号也是不同的。
                dfi[col] = dfi[col].map(self.feature_dict[col])
                dfv[col] = 1.0

        feature_index = dfi.values.tolist()
        feature_val = dfv.values.tolist()
        self.field_size = len(feature_index[0])
        del dfi, dfv
        gc.collect()

        return feature_index, feature_val

In [6]:
dataParse = DataParse(continuous_feature=continuous_feature, category_feature=category_feature)
dataParse.FeatureDictionary(X_trian, X_valid)
train_feature_index, train_feature_val = dataParse.parse(X_trian)
valid_feature_index, valid_feature_val = dataParse.parse(X_valid)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
print('feature_num', dataParse.feature_size)
print('field_num', dataParse.field_size)

feature_num 2605299
field_num 39


## Persistent

In [8]:
y_train.to_csv('../../data/criteo_data/train_y.txt', header=None, index=False)
y_valid.to_csv('../../data/criteo_data/valid_y.txt', header=None, index=False)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [4]:
train_feature_index = pd.DataFrame(train_feature_index)
train_feature_index.to_csv('../../data/criteo_data/train_index.txt', header=None, index=False, sep='\t')

train_feature_val = pd.DataFrame(train_feature_val)
train_feature_val.to_csv('../../data/criteo_data/train_value.txt', header=None, index=False, sep='\t')

valid_feature_index = pd.DataFrame(valid_feature_index)
valid_feature_index.to_csv('../../data/criteo_data/valid_index.txt', header=None, index=False, sep='\t')

valid_feature_val = pd.DataFrame(valid_feature_val)
valid_feature_val.to_csv('../../data/criteo_data/valid_value.txt', header=None, index=False, sep='\t')

# Model

In [2]:
BATCH_SIZE = 256

In [3]:
def get_batch_dataset(label_path, idx_path, value_path):
    label = tf.data.TextLineDataset(label_path)
    idx = tf.data.TextLineDataset(idx_path)
    value = tf.data.TextLineDataset(value_path)

    label = label.map(lambda x: tf.strings.to_number(tf.strings.split(x, sep='\t')), num_parallel_calls=12)
    idx = idx.map(lambda x: tf.strings.to_number(tf.strings.split(x, sep='\t')), num_parallel_calls=12)
    value = value.map(lambda x: tf.strings.to_number(tf.strings.split(x, sep='\t')), num_parallel_calls=12)

    batch_dataset = tf.data.Dataset.zip((label, idx, value))
    batch_dataset = batch_dataset.shuffle(buffer_size=2048)
    batch_dataset = batch_dataset.batch(BATCH_SIZE)
    batch_dataset = batch_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return batch_dataset

In [4]:
class PNN_layer(tf.keras.Model):

    def __init__(self, num_feat, num_field, dropout_deep, deep_layer_sizes, product_layer_dim=10,
                 reg_l1=0.01, reg_l2=1e-5, embedding_size=10, product_type='outer'):
        super().__init__()   # Python2 下使用 super(PNN_layer, self).__init__()
        self.reg_l1 = reg_l1
        self.reg_l2 = reg_l2
        self.num_feat = num_feat                                   # Denoted as
        self.num_field = num_field                                 # Denoted as N
        self.product_layer_dim = product_layer_dim                 # Denoted as D1
        self.dropout_deep = dropout_deep

        # Embedding
        self.feat_embeddings = tf.keras.layers.Embedding(num_feat, embedding_size, embeddings_initializer='uniform')

        initializer = tf.initializers.GlorotUniform()

        # linear part
        self.linear_weights = tf.Variable(initializer(shape=(product_layer_dim, num_field, embedding_size))) # D1*N*M

        # quadratic part
        self.product_type = product_type
        if product_type == 'inner':
            self.theta = tf.Variable(initializer(shape=(product_layer_dim, num_field)))  # D1 * N
        else:
            self.quadratic_weights = tf.Variable(initializer(shape=(product_layer_dim, embedding_size,
                                                                    embedding_size)))   # D1 * M * M
       
        self.product_bias = tf.Variable(tf.random.normal(shape=[product_layer_dim]))
        
        # fc layer
        self.deep_layer_sizes = deep_layer_sizes

        # 神经网络方面的参数
        for i in range(len(deep_layer_sizes)):
            setattr(self, 'dense_' + str(i), tf.keras.layers.Dense(deep_layer_sizes[i]))
            setattr(self, 'batchNorm_' + str(i), tf.keras.layers.BatchNormalization())
            setattr(self, 'activation_' + str(i), tf.keras.layers.Activation('relu'))
            setattr(self, 'dropout_' + str(i), tf.keras.layers.Dropout(dropout_deep[i]))

        # last layer
        self.fc = tf.keras.layers.Dense(1, activation=None, use_bias=True)

    def call(self, feat_index, feat_value):
        # embedding part
        feat_embedding = self.feat_embeddings(feat_index)          # Batch * N * M

        # linear part
        lz = tf.einsum('bnm,dnm->bd', feat_embedding, self.linear_weights)  # Batch * D1

        # quadratic part
        if self.product_type == 'inner':
            theta = tf.einsum('bnm,dn->bdnm', feat_embedding, self.theta)   # Batch * D1 * N * M
            lp = tf.einsum('bdnm,bdnm->bd', theta, theta)
        else:
            embed_sum = tf.reduce_sum(feat_embedding, axis=1)
            p = tf.einsum('bm,bn->bmn', embed_sum, embed_sum)
            lp = tf.einsum('bmn,dmn->bd', p, self.quadratic_weights)  # Batch * D1

        #y_deep = tf.concat((lz, lp), axis=1)
        y_deep = tf.add(lz, lp)
        y_deep = tf.nn.relu(tf.add(y_deep, self.product_bias))
        y_deep = tf.keras.layers.Dropout(self.dropout_deep[0])(y_deep)

        for i in range(len(self.deep_layer_sizes)):
            y_deep = getattr(self, 'dense_' + str(i))(y_deep)
            y_deep = getattr(self, 'batchNorm_' + str(i))(y_deep)
            y_deep = getattr(self, 'activation_' + str(i))(y_deep)
            y_deep = getattr(self, 'dropout_' + str(i))(y_deep)

        output = self.fc(y_deep)
        return output

In [5]:
def cross_entropy_loss(y_true, y_pred):
    return tf.reduce_mean(tf.losses.binary_crossentropy(y_true, y_pred))

In [6]:
def train_one_step(model, optimizer, idx, value, label):
    with tf.GradientTape() as tape:
        output = model(idx, value)
        loss = cross_entropy_loss(y_true=label, y_pred=output)

        reg_loss = []
        for p in model.trainable_variables:
            reg_loss.append(tf.nn.l2_loss(p))
        reg_loss = tf.reduce_sum(tf.stack(reg_loss))
        loss = loss + model.reg_l2 * reg_loss

    grads = tape.gradient(loss, model.trainable_variables)
    grads = [tf.clip_by_norm(g, 100) for g in grads]
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
    return loss

In [7]:
def train_model(model, train_batch_dataset, optimizer, epoch):
    for batch_idx, (label, idx, value) in enumerate(train_batch_dataset):
        if len(label) == 0:
            break

        loss = train_one_step(model, optimizer, idx, value, label)

        if batch_idx % 100 == 0:
            print('Train Epoch:{}, Step:{}, Loss:{:.6f}'.format(epoch, batch_idx, loss.numpy()))

In [8]:
def test_model(model, test_batch_dataset):
    pred_y, true_y = [], []
    binaryloss = tf.keras.metrics.BinaryCrossentropy()
    for batch_idx, (label, idx, value) in enumerate(test_batch_dataset):
        if len(label) == 0:
            break

        output = model(idx, value)
        binaryloss.update_state(y_true=label, y_pred=output)
        pred_y.extend(list(output.numpy()))
        true_y.extend(list(label.numpy()))
    print('Roc AUC: %.5f' % roc_auc_score(y_true=np.array(true_y), y_score=np.array(pred_y)))
    print('LogLoss: %.5f' % binaryloss.result())

In [9]:
pnn = PNN_layer(num_feat=2605299, num_field=39, dropout_deep=[0.5, 0.5, 0.5],
                deep_layer_sizes=[32, 23], product_layer_dim=10,
                reg_l1=0.01, reg_l2=1e-5, embedding_size=10, product_type='outer')

In [10]:
train_label_path='../../data/criteo_data/train_y.txt'
train_idx_path='../../data/criteo_data/train_index.txt'
train_value_path='../../data/criteo_data/train_value.txt'

valid_label_path='../../data/criteo_data/valid_y.txt'
valid_idx_path='../../data/criteo_data/valid_index.txt'
valid_value_path='../../data/criteo_data/valid_value.txt'

In [11]:
train_batch_dataset = get_batch_dataset(train_label_path, train_idx_path, train_value_path)
test_batch_dataset = get_batch_dataset(valid_label_path, valid_idx_path, valid_value_path)

In [12]:
%%time
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
for epoch in range(5):
    train_model(pnn, train_batch_dataset, optimizer, epoch)

Train Epoch:0, Step:0, Loss:1.001947
Train Epoch:0, Step:100, Loss:0.573534
Train Epoch:0, Step:200, Loss:0.498170
Train Epoch:0, Step:300, Loss:0.542426
Train Epoch:0, Step:400, Loss:0.538103
Train Epoch:0, Step:500, Loss:0.485638
Train Epoch:0, Step:600, Loss:0.498241
Train Epoch:0, Step:700, Loss:0.533299
Train Epoch:0, Step:800, Loss:0.494329
Train Epoch:0, Step:900, Loss:0.484494
Train Epoch:0, Step:1000, Loss:0.514401
Train Epoch:0, Step:1100, Loss:0.460145
Train Epoch:0, Step:1200, Loss:0.510576
Train Epoch:0, Step:1300, Loss:0.534661
Train Epoch:0, Step:1400, Loss:0.497923
Train Epoch:0, Step:1500, Loss:0.474125
Train Epoch:0, Step:1600, Loss:0.538227
Train Epoch:0, Step:1700, Loss:0.521797
Train Epoch:0, Step:1800, Loss:0.526401
Train Epoch:0, Step:1900, Loss:0.472369
Train Epoch:0, Step:2000, Loss:0.517725
Train Epoch:0, Step:2100, Loss:0.506703
Train Epoch:0, Step:2200, Loss:0.566899
Train Epoch:0, Step:2300, Loss:0.534414
Train Epoch:0, Step:2400, Loss:0.488129
Train Epoch:

Train Epoch:3, Step:3000, Loss:0.464495
Train Epoch:3, Step:3100, Loss:0.489630
Train Epoch:3, Step:3200, Loss:0.479137
Train Epoch:3, Step:3300, Loss:0.489545
Train Epoch:3, Step:3400, Loss:0.467704
Train Epoch:3, Step:3500, Loss:0.553719
Train Epoch:3, Step:3600, Loss:0.446702
Train Epoch:3, Step:3700, Loss:0.485021
Train Epoch:3, Step:3800, Loss:0.524043
Train Epoch:3, Step:3900, Loss:0.506648
Train Epoch:3, Step:4000, Loss:0.461460
Train Epoch:3, Step:4100, Loss:0.491180
Train Epoch:3, Step:4200, Loss:0.502471
Train Epoch:3, Step:4300, Loss:0.480197
Train Epoch:3, Step:4400, Loss:0.539509
Train Epoch:3, Step:4500, Loss:0.455363
Train Epoch:3, Step:4600, Loss:0.514832
Train Epoch:3, Step:4700, Loss:0.437973
Train Epoch:3, Step:4800, Loss:0.463308
Train Epoch:3, Step:4900, Loss:0.629165
Train Epoch:3, Step:5000, Loss:0.565100
Train Epoch:3, Step:5100, Loss:0.456282
Train Epoch:3, Step:5200, Loss:0.473335
Train Epoch:3, Step:5300, Loss:0.469686
Train Epoch:3, Step:5400, Loss:0.502621


In [13]:
test_model(pnn, test_batch_dataset)

Roc AUC: 0.75408
LogLoss: 0.48647
